In [3]:
import numpy as np

def cannon_matrix_multiply(A, B, grid_dim):
    """Perform matrix multiplication using Cannon's algorithm.

    Args:
        A: Matrix A (np.array of shape (N,N))
        B: Matrix B (np.array of shape (N,N))
        grid_dim: Number of processors per row/column in the processor grid

    Returns:
        C: Resulting matrix from A @ B
    """
    N = A.shape[0]
    assert A.shape == B.shape and N % grid_dim == 0, "Invalid matrix size or grid"

    block_size = N // grid_dim
    C = np.zeros_like(A, dtype=float)

    # Divide matrices into blocks for each processor
    A_blocks = np.zeros((grid_dim, grid_dim, block_size, block_size))
    B_blocks = np.zeros((grid_dim, grid_dim, block_size, block_size))
    
    # Initialize blocks
    for i in range(grid_dim):
        for j in range(grid_dim):
            A_blocks[i, j] = A[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
            B_blocks[i, j] = B[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]

    # Initial skewing
    # Shift row i of A left by i positions
    for i in range(grid_dim):
        A_blocks[i] = np.roll(A_blocks[i], -i, axis=0)
    
    # Shift column j of B up by j positions  
    for j in range(grid_dim):
        B_blocks[:, j] = np.roll(B_blocks[:, j], -j, axis=0)

    # Main computation loop
    for step in range(grid_dim):
        # Each processor computes its local matrix multiplication
        for i in range(grid_dim):
            for j in range(grid_dim):
                C[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size] += \
                    A_blocks[i, j] @ B_blocks[i, j]

        # Shift all rows of A left by 1 position (except in the last iteration)
        if step < grid_dim - 1:
            for i in range(grid_dim):
                A_blocks[i] = np.roll(A_blocks[i], -1, axis=0)
            
            # Shift all columns of B up by 1 position
            for j in range(grid_dim):
                B_blocks[:, j] = np.roll(B_blocks[:, j], -1, axis=0)

    return C

In [9]:
N = 8
P = 4  # 2x2 processor grid

# Use fixed values for reproducible testing
np.random.seed(42)
A = np.arange(N*N).reshape(N, N)
B = np.arange(N*N).reshape(N, N)


C = cannon_matrix_multiply(A, B, P)

C_ref = np.matmul(A, B)

np.testing.assert_allclose(C, C_ref, atol=1e-10)
print("SUCCESS: C == C_ref")

SUCCESS: C == C_ref


In [10]:
print(A)

[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]
 [24 25 26 27 28 29 30 31]
 [32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47]
 [48 49 50 51 52 53 54 55]
 [56 57 58 59 60 61 62 63]]


In [32]:
A_blocks[0,1]

array([[ 2.,  3.],
       [10., 11.]])

In [13]:
grid_dim = P
block_size = N // grid_dim
print(f"block_size: {block_size}")

# Divide matrices into blocks for each processor
A_blocks = np.zeros((grid_dim, grid_dim, block_size, block_size))
B_blocks = np.zeros((grid_dim, grid_dim, block_size, block_size))

for i in range(grid_dim):
    for j in range(grid_dim):
        A_blocks[i, j] = A[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]
        B_blocks[i, j] = B[i*block_size:(i+1)*block_size, j*block_size:(j+1)*block_size]


block_size: 2


In [23]:
# Shift column j of B up by j positions  
for j in range(grid_dim):
    B_blocks[:, j] = np.roll(B_blocks[:, j], -j, axis=0)

In [26]:
B_blocks[1]

array([[[16., 17.],
        [24., 25.]],

       [[34., 35.],
        [42., 43.]],

       [[52., 53.],
        [60., 61.]],

       [[ 6.,  7.],
        [14., 15.]]])

In [18]:
for i in range(grid_dim):
    A_blocks[i] = np.roll(A_blocks[i], -i, axis=0)

In [25]:
A_blocks[3]

array([[[54., 55.],
        [62., 63.]],

       [[48., 49.],
        [56., 57.]],

       [[50., 51.],
        [58., 59.]],

       [[52., 53.],
        [60., 61.]]])

array([[0., 1.],
       [8., 9.]])